# ssGSEA analysis of t-stastic derived from differential expression analysis using ConsensusTME

Script related to figure 3h

### Aim:
- Calculate NES of consensusTME gene sets using t-statistic derived from differential expression analysis

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

## Load differential expression data

In [3]:
data <- read.csv('../../Data/3/Diff_exp_highlow_tum_cell_patientdependent.txt',
                 sep='\t',
                 header=T,
                 row.names='Hugo_symbol')

head(data)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
CYBB,-1.870505,7.517236,-7.383829,9.506638e-09,0.0001394434,9.284987
VAMP5,-1.638811,7.159211,-5.762822,1.374465e-06,0.0100803241,5.020438
APOC1,-2.131354,12.821060,-5.479986,3.302432e-06,0.0105828459,4.257513
IL7R,-3.111353,10.072070,-5.426074,3.902423e-06,0.0105828459,4.111945
NKG7,-1.178869,6.407097,-5.360814,4.775790e-06,0.0105828459,3.935731
CD53,-1.710032,9.386776,-5.338679,5.114235e-06,0.0105828459,3.875965


### Select t-statistic

In [4]:
data_t <- data['t']

head(data_t)

,t
CYBB,-7.383829
VAMP5,-5.762822
APOC1,-5.479986
IL7R,-5.426074
NKG7,-5.360814
CD53,-5.338679


In [5]:
data_mtx <- as.matrix(data_t)

## Load ConsensusTME gene sets

In [6]:
ConTME_GeneSets <- read.csv('../../Data/3/ConsensusTME_GeneSets.txt',
                            sep='\t',
                            header=T)

head(ConTME_GeneSets)

B_cells,Cytotoxic_cells,Dendritic_cells,Endothelial,Eosinophils,Fibroblasts,Macrophages,Macrophages_M1,Macrophages_M2,Mast_cells,Monocytes,NK_cells,Neutrophils,Plasma_cells,T_cells_CD4,T_cells_CD8,T_cells_gamma_delta,T_regulatory_cells
BLNK,CTSW,BCL2A1,CD93,ABHD2,ABCA6,ACTR2,ACTR2,AIF1,ATP6V1C1,AIF1,APOBEC3G,BCL2A1,CD27,AIM2,BCL11B,APOBEC3G,CCR7
CD1C,GZMA,BIRC3,CDH5,BCL2A1,ASPN,ATG7,CCL19,ATP6V1C1,CCL4,AKAP13,CCL4,C5AR1,CD38,ARHGAP15,BIN2,CCL4,CD2
CD37,GZMB,CCL19,EMCN,CCL4,COL3A1,ATP6V1C1,CCL4,CLEC10A,CD84,C5AR1,CCR5,CSF2RB,CYBA,BATF,CCL4,CCR2,CD247
CD53,GZMH,CCL4,ERG,CCR1,DCN,BCL2A1,CCR1,CLEC4A,LCP2,CCR2,CD2,CSF3R,ENTPD1,BCL11B,CCR5,CCR5,CD27
CD69,KLRB1,CCR2,KDR,CD69,GREM1,C5AR1,CCR7,FGR,MMP9,CD244,CD244,FCGR3B,HSPA6,CCL4,CCR7,CD2,CD37
CD80,KLRK1,CCR5,PCDH12,CSF2RB,ISLR,CCL19,CD38,HCK,SAMSN1,CD37,CD247,FPR1,MTDH,CCR2,CD2,CD244,CD3E


In [7]:
ConTME_GeneSets <- convertRowsToList(t(ConTME_GeneSets))

## Run ssGSEA

In [8]:
con_NES <- gsva(data_mtx,
                ConTME_GeneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“14668 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 18 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [9]:
con_NES <- data.frame("Term"=rownames(con_NES),
                      con_NES)

rownames(con_NES) <- NULL

head(con_NES)

Term,t
B_cells,-1.671425
Cytotoxic_cells,-1.858738
Dendritic_cells,-2.114059
Endothelial,-1.740926
Eosinophils,-1.811567
Fibroblasts,-1.626924


In [10]:
colnames(con_NES)[2] <- "ConsensusTME_NES"

In [11]:
write.table(con_NES,
            file='../../Data/3/ConsensusTME_NES_after_DiffExp.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script